<a href="https://colab.research.google.com/github/rizkintaa/Learn-n-Learn/blob/main/Prediksi_angka_kemiskinan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
# ===============================================================
# PROYEK PEMBELAJARAN MESIN:
# Prediksi Tingkat Kemiskinan di Indonesia (Random Forest)
# ===============================================================

# Import library utama
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# ===============================================================
# BACA DAN PERSIAPKAN DATA
# ===============================================================

# Baca data indikator sosial ekonomi
df = pd.read_excel('data_kemiskinan_2025_2.xlsx')

# Bersihkan nama kolom dari spasi
df.columns = df.columns.str.strip()

# Tampilkan nama kolom untuk verifikasi
print("Kolom yang terbaca:", df.columns.tolist())

# Pastikan kolom numerik
num_cols = ['PDRB_perkapita', 'Pengangguran', 'IPM',
            'Lama_sekolah', 'Air_bersih', 'Listrik', 'Jumlah_Penduduk']

# Ubah satuan jumlah penduduk dari ribu → juta
df['Jumlah_Penduduk'] = df['Jumlah_Penduduk'] / 1000

# Pastikan semua kolom numerik
df[num_cols] = df[num_cols].astype(float)

# ===============================================================
# TAMBAHKAN DATA TARGET (Persentase Kemiskinan)
# ===============================================================
# Baca file target dari BPS
df_target = pd.read_csv('persentase_miskin.csv')  # pastikan kolom: Provinsi, Tahun, Persentase_miskin

# Gabungkan berdasarkan Provinsi & Tahun
df = df.merge(df_target, on=['Provinsi', 'Tahun'])

print("\n Dataset akhir setelah digabung:")

# tampilkan semua isi dataset
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')

print(df.to_string(index=False))

# ===============================================================
# CEK & BERSIHKAN DATA
# ===============================================================
# Hapus baris kosong jika ada
df = df.dropna()

# Konversi semua kolom numerik jika ada string
df[num_cols + ['Persentase_miskin']] = df[num_cols + ['Persentase_miskin']].apply(pd.to_numeric, errors='coerce')

print("\n Cek nilai kosong setelah pembersihan:")
print(df.isnull().sum())

# ===============================================================
# SIAPKAN DATA UNTUK MODEL
# ===============================================================
X = df[['PDRB_perkapita', 'Pengangguran', 'IPM', 'Lama_sekolah',
        'Air_bersih', 'Listrik', 'Jumlah_Penduduk']]
y = df['Persentase_miskin']

# Bagi data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ===============================================================
# TRAINING MODEL RANDOM FOREST
# ===============================================================
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    random_state=42
)
rf.fit(X_train, y_train)

# ===============================================================
# EVALUASI MODEL
# ===============================================================
y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5

print("\n Evaluasi Model Random Forest:")
print(f"R² Score : {r2:.3f}")
print(f"RMSE     : {rmse:.3f}")

# ===============================================================
# VISUALISASI HASIL
# ===============================================================

# --- Scatter Plot Prediksi vs Aktual ---
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel('Aktual (%)')
plt.ylabel('Prediksi (%)')
plt.title('Prediksi vs Aktual Tingkat Kemiskinan (Random Forest)')
plt.grid(True)
plt.show()

# --- Feature Importance ---
importance = pd.DataFrame({
    'Faktor': X.columns,
    'Pentingnya': rf.feature_importances_
}).sort_values(by='Pentingnya', ascending=False)

plt.figure(figsize=(8,4))
sns.barplot(data=importance, x='Pentingnya', y='Faktor', hue='Faktor', palette='mako', legend=False)
plt.title('Urutan Faktor yang Mempengaruhi Tingkat Kemiskinan di Indonesia')
plt.xlabel('Tingkat Kepentingan')
plt.ylabel('Faktor')
plt.show()

print("\n Urutan Faktor Paling Berpengaruh:")
print(importance)

# ===============================================================
# END OF CODE
# ===============================================================

# ===============================================================
# PREDIKSI TINGKAT KEMISKINAN TAHUN 2026
# ===============================================================

# Duplikasi data tahun 2025 untuk diproyeksikan ke 2026
df_2026 = df.copy()
df_2026['Tahun'] = 2026

# Estimasi perubahan indikator (perkiraan moderat)
df_2026['PDRB_perkapita'] *= 1.05      # naik 5%
df_2026['IPM'] *= 1.01                 # naik 1%
df_2026['Listrik'] *= 1.005            # naik 0.5%
df_2026['Air_bersih'] *= 1.01          # naik 1%
df_2026['Pengangguran'] *= 0.98        # turun 2%
df_2026['Lama_sekolah'] += 0.1         # naik 0.1 tahun
df_2026['Jumlah_Penduduk'] *= 1.012    # naik 1.2%

# Siapkan fitur untuk prediksi
X_2026 = df_2026[['PDRB_perkapita', 'Pengangguran', 'IPM', 'Lama_sekolah',
                  'Air_bersih', 'Listrik', 'Jumlah_Penduduk']]

# Gunakan model Random Forest yang sudah dilatih untuk prediksi
df_2026['Prediksi_Kemiskinan_2026'] = rf.predict(X_2026)

# Tampilkan hasil
print("\n Prediksi Tingkat Kemiskinan Tahun 2026 per Provinsi:")
print(df_2026[['Provinsi', 'Prediksi_Kemiskinan_2026']].sort_values(by='Prediksi_Kemiskinan_2026'))

# Simpan ke file Excel
df_2026[['Provinsi', 'Tahun', 'Prediksi_Kemiskinan_2026']].to_excel('prediksi_kemiskinan_2026.xlsx', index=False)
print("\n Hasil prediksi 2026 disimpan ke 'prediksi_kemiskinan_2026.xlsx'")

# Visualisasi hasil prediksi 2026
plt.figure(figsize=(10,6))
sns.barplot(data=df_2026.sort_values('Prediksi_Kemiskinan_2026', ascending=False),
            x='Prediksi_Kemiskinan_2026', y='Provinsi', palette='crest')
plt.title('Prediksi Tingkat Kemiskinan per Provinsi (Tahun 2026)')
plt.xlabel('Persentase Kemiskinan (%)')
plt.ylabel('Provinsi')
plt.show()

import joblib
joblib.dump(rf, 'model_kemiskinan_rf.pkl')
print("✅ Model berhasil disimpan ke model_kemiskinan_rf.pkl")

